## Introduction: Business Problem

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Indian restaurant** in **Dubai**, UAE.

Since there are lots of restaurants in Berlin we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Indian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Dubai center will be obtained using **Google Maps API geocoding** of well known Dubai location (Dubai Mall)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Dubai Mall.

Let's first find the latitude & longitude of Dubai Mall, using specific, well known address and Google Maps geocoding API.

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



In [5]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [31]:
import requests

address = 'Dubai Mall'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

25.1970924 55.2790356376064


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Dubai Mall. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).


In [9]:
search_query = 'Indian'
radius = 5000
print(search_query + ' .... OK!')

Indian .... OK!


In [32]:
CLIENT_ID = 'F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ' # your Foursquare ID
CLIENT_SECRET = 'QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ
CLIENT_SECRET:QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152


In [33]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ&client_secret=QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152&ll=25.1970924,55.2790356376064&v=20180604&query=Indian&radius=5000&limit=30'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d10b82aad1789002ccd705f'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'primary': True,
      'shortName': 'Indian'}],
    'hasPerk': False,
    'id': '5be56ba5646e38002c6fb889',
    'location': {'address': 'Dubai Design District D3',
     'cc': 'AE',
     'city': 'دبي',
     'country': 'الإمارات العربية المتحدة',
     'distance': 2159,
     'formattedAddress': ['Dubai Design District D3',
      'دبي',
      'الإمارات العربية المتحدة'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.18795,
       'lng': 55.297947}],
     'lat': 25.18795,
     'lng': 55.297947,
     'neighborhood': "Za'abeel",
     'state': 'دبي'},
    'name': 'Mohalla Indian Street Food',
    'referralId': 'v-1561376810'},
   {'categories': [{'icon':

In [35]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.state,name,referralId
0,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,5be56ba5646e38002c6fb889,Dubai Design District D3,AE,دبي,الإمارات العربية المتحدة,NaN,2159,"[Dubai Design District D3, دبي, الإمارات العرب...","[{'lat': 25.18795, 'label': 'display', 'lng': ...",25.187950,55.297947,Za'abeel,دبي,Mohalla Indian Street Food,v-1561376810
1,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,4b99106ff964a520fb5e35e3,The Dubai Mall,AE,دبي,الإمارات العربية المتحدة,NaN,245,"[The Dubai Mall, دبي, الإمارات العربية المتحدة]","[{'lat': 25.196720033922475, 'label': 'display...",25.196720,55.276636,NaN,دبي,India Palace Express,v-1561376810
2,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,548130c6498e82fa71d319e4,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,1660,"[دبي, الإمارات العربية المتحدة]","[{'lat': 25.210377436632708, 'label': 'display...",25.210377,55.271535,NaN,دبي,Metro Grand Indian Restaurant,v-1561376810
3,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,5a0f13df1755622302f4fe6d,Wasl Square,AE,دبي,الإمارات العربية المتحدة,Safa Park Road,4022,"[Wasl Square (Safa Park Road), دبي, الإمارات ا...","[{'lat': 25.185643, 'label': 'display', 'lng':...",25.185643,55.241163,NaN,دبي,Lazeez Indian Cuisine,v-1561376810
4,"[{'icon': {'suffix': '.png', 'prefix': 'https:...",False,5016c225e4b0ad6ab3c135f0,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,3103,[الإمارات العربية المتحدة],"[{'lat': 25.176758, 'label': 'display', 'lng':...",25.176758,55.257954,NaN,NaN,Indian Handy Craft,v-1561376810


In [36]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,state,id
0,Mohalla Indian Street Food,Indian Restaurant,Dubai Design District D3,AE,دبي,الإمارات العربية المتحدة,NaN,2159,"[Dubai Design District D3, دبي, الإمارات العرب...","[{'lat': 25.18795, 'label': 'display', 'lng': ...",25.187950,55.297947,Za'abeel,دبي,5be56ba5646e38002c6fb889
1,India Palace Express,Indian Restaurant,The Dubai Mall,AE,دبي,الإمارات العربية المتحدة,NaN,245,"[The Dubai Mall, دبي, الإمارات العربية المتحدة]","[{'lat': 25.196720033922475, 'label': 'display...",25.196720,55.276636,NaN,دبي,4b99106ff964a520fb5e35e3
2,Metro Grand Indian Restaurant,Asian Restaurant,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,1660,"[دبي, الإمارات العربية المتحدة]","[{'lat': 25.210377436632708, 'label': 'display...",25.210377,55.271535,NaN,دبي,548130c6498e82fa71d319e4
3,Lazeez Indian Cuisine,Indian Restaurant,Wasl Square,AE,دبي,الإمارات العربية المتحدة,Safa Park Road,4022,"[Wasl Square (Safa Park Road), دبي, الإمارات ا...","[{'lat': 25.185643, 'label': 'display', 'lng':...",25.185643,55.241163,NaN,دبي,5a0f13df1755622302f4fe6d
4,Indian Handy Craft,Arts & Crafts Store,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,3103,[الإمارات العربية المتحدة],"[{'lat': 25.176758, 'label': 'display', 'lng':...",25.176758,55.257954,NaN,NaN,5016c225e4b0ad6ab3c135f0
5,Gazebo Indian cuisine DIFC,Indian Restaurant,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,2290,[الإمارات العربية المتحدة],"[{'lat': 25.217665, 'label': 'display', 'lng':...",25.217665,55.278635,NaN,NaN,56df0c60498e77585c2fb553
6,Jashan Indian Restaurant,Indian Restaurant,Sheikh Zayed Road,AE,دبي,الإمارات العربية المتحدة,NaN,2887,"[Sheikh Zayed Road, دبي, الإمارات العربية المت...","[{'lat': 25.222857778279415, 'label': 'display...",25.222858,55.282374,NaN,دبي,4c2b70792219c928c791a448
7,Indian Movie Night!,Other Nightlife,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,5295,"[دبي, الإمارات العربية المتحدة]","[{'lat': 25.17967670518014, 'label': 'display'...",25.179677,55.230115,NaN,دبي,4eeb67842c5beea6206a198e
8,Indian Barber Shop,Salon / Barbershop,Karama Bus Station 7333,AE,دبي,الإمارات العربية المتحدة,Al Karama Fish Market,5567,[Karama Bus Station 7333 (Al Karama Fish Marke...,"[{'lat': 25.24253264752241, 'label': 'display'...",25.242533,55.302122,NaN,دبي,4f3755dae4b0a1f196cb3b16
9,The Indian Pavillion,Indian Restaurant,Safa Spinneys,AE,دبي,الإمارات العربية المتحدة,NaN,6045,"[Safa Spinneys, دبي, الإمارات العربية المتحدة]","[{'lat': 25.16905078692849, 'label': 'display'...",25.169051,55.227643,NaN,دبي,4b5beabef964a520261d29e3


In [37]:
dataframe_filtered.name

0                            Mohalla Indian Street Food
1                                  India Palace Express
2                         Metro Grand Indian Restaurant
3                                 Lazeez Indian Cuisine
4                                    Indian Handy Craft
5                            Gazebo Indian cuisine DIFC
6                              Jashan Indian Restaurant
7                                   Indian Movie Night!
8                                    Indian Barber Shop
9                                  The Indian Pavillion
10             Indian passport and visa services center
11                                        Indian Summer
12                                Our Own Indian School
13                         Little Hut Indian Restaurant
14                                 Sapne Indian Cuisine
15    Pathans South Indian Restaurant, Satwa Round-a...
16                   Global indian international School
17                   Temptation Indian Grills & 

In [38]:
dataframe_filtered.shape

(20, 15)

In [39]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the 

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='JLT',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


# add the Indian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.1
    ).add_to(venues_map)


# display map
venues_map